<a href="https://colab.research.google.com/github/itchy-squid/AdventOfCode/blob/main/2022/22/AdventOfCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import re

DIRECTIONS = [(1,0),(0,1),(-1,0),(0,-1)]

class Tile:
  WALL = '#'
  PATH = '.'
  VOID = ' '

class MonkeyMap:
  def __init__(self, mm):
    self._height = len(mm)
    self._width = max([len(l) for l in mm])
    self._map = mm

  def height(self): return self._height
  def width(self): return self._width
  def starting_position(self): return (self._map[0].index(Tile.PATH),0)

  def tile(self, p): return [*(self._map[p[1]][p[0]:]),' '][0]

  def __next_position(self, p, d):
    return ((p[0]+d[0])%self.width(),(p[1]+d[1])%self.height())

  def what_if(self, p0, d):
    p1 = self.__next_position(p0,d)
    match self.tile(p1):
      case Tile.PATH: return p1
      case Tile.WALL: return p0
      case Tile.VOID:
        while self.tile(p1) == Tile.VOID:
          p1 = self.__next_position(p1,d)
        return p1 if self.tile(p1) == Tile.PATH else p0

class Person:
  def __init__(self, pos):
    self._d_i = 0
    self.position = pos

  def direction(self): return DIRECTIONS[self._d_i]
  def turn(self, inc): self._d_i = (self._d_i + inc) % len(DIRECTIONS)

class Turn:
  RIGHT = 'R'
  LEFT = 'L'

class Brain:
  def __init__(self, body, instructions):
    self._body = body
    self._instructions = instructions

  def move(self, map):
    #print(self._instructions)
    now = re.search(f'(\d+|{Turn.RIGHT}|{Turn.LEFT})', self._instructions).group()
    self._instructions = self._instructions[len(now):]

    match now:
      case Turn.RIGHT: self._body.turn(1)
      case Turn.LEFT: self._body.turn(-1)
      case _:
        for i in range(0,int(now)):
          self._body.position = map.what_if(
              self._body.position,
              self._body.direction())

    return self._instructions


In [25]:
from google.colab import files
with open('2022-22-1.txt','r') as f:
  lines = f.read().splitlines()

mmap = MonkeyMap(lines[:-2])
body = Person(mmap.starting_position())
brain = Brain(body, lines[-1])

def printScene ():
  for y in range(0,mmap.height()):
    print(''.join(['X' if (x,y) in visited
                   else mmap.tile((x,y)) for x in range(0,mmap.width())]))

visited = {body.position}
while brain.move(mmap):
  visited |= {body.position}
  #printScene()
  #print()

row,col=body.position[1]+1,body.position[0]+1
print((1000*row)+(4*col)+DIRECTIONS.index(body.direction()))

106094
